# Generative Adversarial Networks (GANs) — Practical Assignment Notebook

## Introduction
In this notebook, you will practice building and modifying different GAN architectures to understand how image generation works in deep learning. You will start with a basic GAN, then move to DCGAN, Conditional GANs, and finally apply them to new datasets. Each task is designed to strengthen your understanding through hands-on experimentation.

By the end of this notebook, you will be able to:
- Build and train GAN and DCGAN models
- Understand how convolutional layers improve image generation
- Control generated outputs using Conditional GANs
- Generalize models to new datasets
- Interactively generate images based on user input

---

## Task 1 — Basic GAN Modification (Easy)
### Goal
Modify the basic GAN model to generate **32×32** images instead of **28×28**.

### Steps
1. Change the Generator’s last Linear layer to output **1024** values instead of **784**.
2. Update the reshaping step to return images of shape **(1, 32, 32)**.
3. Modify the Discriminator input layer to accept **1024** features.
4. Train the model and visualize generated samples.

### Deliverable
- Generated 32×32 images
- Short explanation of what you changed

### What you learn
Understanding how architectural dimensions affect model outputs.

---

## Task 2 — DCGAN Filter Experiment (Medium)
### Goal
Experiment with different convolution filter sizes in DCGAN.

### Steps
1. Change Discriminator filters from:
   - Conv2d(1 → 64) → Conv2d(64 → 128)
   to:
   - Conv2d(1 → 32) → Conv2d(32 → 64)
2. Retrain the DCGAN.
3. Compare image quality and training stability.

### Deliverable
- Generated images for both configurations
- Short comparison paragraph

### What you learn
Effect of channel capacity on feature learning and image quality.

---

## Task 3 — Conditional Generation Test (Medium)
### Goal
Use Conditional DCGAN to generate specific MNIST digits on demand.

### Steps
1. Train a Conditional DCGAN on MNIST.
2. Generate one image for each label (0–9).
3. Display them in a single row.

### Deliverable
- Image grid of requested digits
- Code snippet used for conditional generation

### What you learn
How label conditioning controls the Generator output.

---

## Task 4 — Change Dataset (Advanced)
### Goal
Replace MNIST with **Fashion-MNIST** and retrain DCGAN.

### Steps
1. Replace `datasets.MNIST` with `datasets.FashionMNIST`.
2. Train DCGAN with the same architecture.
3. Visualize generated clothing items.

### Deliverable
- Generated Fashion-MNIST samples
- Short comment on image quality

### What you learn
Generalization of GANs to new datasets.

---



## Task 5 — Conditional Fashion Generator (Advanced)
### Goal
Build a Conditional DCGAN that generates specific Fashion-MNIST clothing items based on user input.

### Fashion-MNIST Label Map
| Label | Clothing Item |
|--------|--------------|
| 0 | T-shirt / Top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

### Steps
1. Train a Conditional DCGAN on Fashion-MNIST.
2. After training, ask the user to input a label (0–9).
3. Generate and display the requested clothing image.

### Inference Example
```python
label = int(input("Enter Fashion label (0–9): "))
z = torch.randn(1, latent_dim).to(device)
label_tensor = torch.tensor([label]).to(device)

with torch.no_grad():
    img = G(z, label_tensor)

plt.imshow(img.squeeze().cpu(), cmap="gray")
plt.title(f"Generated Class: {label}")
plt.axis("off")
```

### Deliverable
- Generated images for labels 0–9
- Screenshot of user-input generation
- Short reflection on which classes are easier or harder to generate

### What you learn
Interactive conditional generation and practical GAN deployment logic.

---

## Final Note
Experiment freely. GAN performance depends heavily on architecture, hyperparameters, and training stability. Your goal is not only to make the model work — but to understand why it behaves the way it does.

**Good luck — and enjoy teaching machines to draw!